In [1]:
# Notebook to extract the ball position from .ant file
import pandas as pd

In [2]:
## Apenas para quem precisa conectar do Drive, não sei se isso é necessário todas as vezes. Espero que sim.
## 
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
JOGO = 'REDSCAT2'

In [4]:
# Ranieri
#df = pd.read_csv('./Projects/futebol/Dados_Futebol/REDSCAT1.ant', sep =('\s+'), header=None)

# Ulisses
#df = pd.read_csv('/content/drive/My Drive/Dados_Futebol/REDSCAT1.ant', sep =('\s+'), header=None)

# Leandro

df = pd.read_csv('jogos\\' + JOGO +'.ant', sep =('\s+'), header=None)

Os arquivos são organizados da seguinte forma:

1ª coluna: frame do momento que a ação técnica foi realizada;
2ª coluna: número do jogador. No caso do futsal, do 1 ao 14 tratam-se de jogadores do São Paulo (Spo) e do 15 ao 28 dos jogadores do Corinthians (todo-poderoso-timão Cor). Já para os dados de futebol, do 1 ao 14 tratam-se de jogadores do Atlético Paranaense (Cap) e do 15 ao 28 dos jogadores do São Paulo.
3ª e 4ª coluna: coordenadas X e Y do jogador que realizou a ação técnica;
5ª coluna: ação técnica realizada. Cada ação recebe um número, a saber:
 
0 - Domínio
1 - Passe
2 - Drible
3 - Finalização-chute
4- Finalização-cabeca
5 - Desarme- parte inferior do corpo
6 - Desarme-superior
7- Defesa-goleiro
8- Fair-play
9- Tiro-de-meta
10 - Lateral
11- Escanteio
12- Tiro-livre
13 - Falta
14 - Gol
15 - Condução  
 
PS1: No caso dos dados de futebol, as seguintes ações são diferentes:
 
8 - Saída de cruzamento do goleiro
12 - Impedimento 
 
PS2: No momento da falta, marcamos para um mesmo frame quem sofreu e quem realizou a falta (uma linha seguida da outra).
 
PS3: No momento do gol, marcamos para um mesmo frame a finalização e o gol (uma linha seguida da outra).
 
6ª coluna: avisa se ação foi certa (1) ou errada (0). Para condução (ação 15), não discriminamos 0 ou 1, ou seja, vai ficar sempre 0. 
 
Qualquer dúvida me escrevam novamente.

In [5]:
df.head()

,0,1,2,3,4,5
0,90,23.0,52.7777,34.4084,1.0,1.0
1,104,24.0,52.4860,37.5700,0.0,1.0
2,105,24.0,52.4860,37.5700,15.0,1.0
3,126,24.0,53.3324,38.5844,1.0,1.0
4,153,25.0,63.2471,45.0185,0.0,1.0


In [6]:
 # add position of the ball on the previous known frame to the row

df['shifted_x'] = df[2].shift()
df['shifted_y'] = df[3].shift()
df['previous_known_frame'] = df[0].shift()

In [7]:
# fix null values (in the end, frame will be removed..)
df['shifted_x'][0]= 1
df['shifted_y'][0]= 1

C:\Users\stiva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\stiva\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#Generate a new, empty dataframe (for posterior use)
df_complete = pd.DataFrame()

In [9]:
df.head(5)

,0,1,2,3,4,5,shifted_x,shifted_y,previous_known_frame
0,90,23.0,52.7777,34.4084,1.0,1.0,1.0000,1.0000,NaN
1,104,24.0,52.4860,37.5700,0.0,1.0,52.7777,34.4084,90.0
2,105,24.0,52.4860,37.5700,15.0,1.0,52.4860,37.5700,104.0
3,126,24.0,53.3324,38.5844,1.0,1.0,52.4860,37.5700,105.0
4,153,25.0,63.2471,45.0185,0.0,1.0,53.3324,38.5844,126.0


In [10]:
curr_row = 1
mapping_row = df['previous_known_frame'][1]
curr_x = df['shifted_x'][curr_row]
curr_y = df['shifted_y'][curr_row]
#Get the numer of unknown rows
number_unknown_rows = df[0][curr_row]- df['previous_known_frame'][curr_row]
#Calculate position change of each 'phantom row' (current_value - shifted_value / number_unknown_rows)
x_to_sum = (df[2][curr_row] - df['shifted_x'][curr_row])/(number_unknown_rows)
y_to_sum = (df[3][curr_row] - df['shifted_y'][curr_row])/(number_unknown_rows)
#Verify the team with the ball through the player number
if (df[1][curr_row]) <= 14:
    possession_team = 0
else: 
    possession_team = 1
#Append rows to an empty dataset, adding "phantom" rows for rows not present in the original
df_complete = df_complete.append([[mapping_row, possession_team, df['shifted_x'][curr_row], df['shifted_y'][curr_row]]])

#Repeat the above procedures for each row
while mapping_row < df[0][df[0].count()-2]:
    mapping_row  = mapping_row + 1
    if df[0][curr_row] <= mapping_row:
        curr_row = curr_row + 1
        number_unknown_rows = df[0][curr_row]-df['previous_known_frame'][curr_row]
        if (df[1][curr_row]) <= 14:
            possession_team = 0
        else: 
            possession_team = 1
        if (number_unknown_rows != 0):
            x_to_sum = (df[2][curr_row] - df['shifted_x'][curr_row])/(number_unknown_rows)
            y_to_sum = (df[3][curr_row] - df['shifted_y'][curr_row])/(number_unknown_rows)
        curr_x =df['shifted_x'][curr_row]
        curr_y = df['shifted_y'][curr_row]
    else:
        curr_x = curr_x + x_to_sum
        curr_y = curr_y + y_to_sum
    df_complete = df_complete.append([[mapping_row, possession_team, curr_x,curr_y]])    
    
    
    

In [11]:
df_complete.head(5)

,0,1,2,3
0,90.0,1,52.777700,34.408400
0,91.0,1,52.756864,34.634229
0,92.0,1,52.736029,34.860057
0,93.0,1,52.715193,35.085886
0,94.0,1,52.694357,35.311714


In [12]:
#Fix indexes
df_complete = df_complete.set_index(0)


In [13]:
#Save results to new file
df_complete.to_csv(JOGO+".csv")